In [15]:
import numpy as np
import pandas as pd
from utils import *

In [16]:
df = load_from_txt('/Users/shamanthk/Documents/iomics/DoWhy-attribution/datasets/sachs.2005.continuous.txt')
df = df.dropna()
df.drop_duplicates(inplace=True)
labels = list(df.columns)
df.head()

,raf,mek,plc,pip2,pip3,erk,akt,pka,pkc,p38,jnk
0,26.4,13.2,8.82,18.30,58.80,6.61,17.0,414.0,17.00,44.9,40.0
1,35.9,16.5,12.30,16.80,8.13,18.60,32.5,352.0,3.37,16.5,61.5
2,59.4,44.1,14.60,10.20,13.00,14.90,32.5,403.0,11.40,31.9,19.5
3,73.0,82.8,23.10,13.50,1.29,5.83,11.8,528.0,13.70,28.6,23.1
4,33.7,19.8,5.19,9.73,24.80,21.10,46.1,305.0,4.66,25.7,81.3


In [20]:
from sklearn.ensemble import IsolationForest

iso_forest = IsolationForest(contamination=0.0005, random_state=42)
df['anomaly'] = iso_forest.fit_predict(df)
df_iso_outliers = df[df['anomaly'] == -1]

In [21]:
import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
df_iso_outliers

,raf,mek,plc,pip2,pip3,erk,akt,pka,pkc,p38,jnk,anomaly
2709,973.0,1499.0,685.0,1241.0,9.06,196.0,1358.0,71.00,704.0,3587.0,914.0,-1
2776,2890.0,4451.0,698.0,1263.0,157.00,55.2,365.0,13.60,1186.0,6153.0,1134.0,-1
2821,122.0,209.0,965.0,1540.0,10.30,160.0,882.0,55.20,799.0,4105.0,2227.0,-1
3177,196.0,334.0,2666.0,4740.0,184.00,93.9,519.0,9.22,717.0,3555.0,3398.0,-1


In [9]:
df_iso_outliers.to_csv('anamoulous_data.csv')

In [23]:
from scipy.stats import zscore

z_scores = df.apply(zscore)
threshold = 5
outliers_zscore = (z_scores.abs() > threshold).any(axis=1)
df_zscore_outliers = df[outliers_zscore]

In [28]:
Q1 = df.quantile(0.005)
Q3 = df.quantile(0.955)
IQR = Q3 - Q1
outliers_iqr = ((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)
df_iqr_outliers = df[outliers_iqr]

In [26]:
df_zscore_outliers

,raf,mek,plc,pip2,pip3,erk,akt,pka,pkc,p38,jnk,anomaly
359,53.3,34.60,2.89,1.11,764.00,8.66,12.7,219.0,24.60,63.80,137.00,1
671,63.2,37.20,37.90,81.30,39.20,316.00,621.0,1472.0,18.10,30.80,9.73,1
748,37.5,16.40,18.60,41.00,5.19,2571.00,3555.0,4491.0,14.60,36.20,24.40,1
1047,67.3,31.30,1.00,7.50,389.00,21.30,25.3,621.0,7.04,22.10,22.10,1
1240,79.1,52.30,12.90,138.00,12.10,172.00,453.0,4826.0,7.64,28.90,28.40,1
...,...,...,...,...,...,...,...,...,...,...,...,...
7001,46.6,1.57,4.37,1.49,382.00,20.50,38.5,483.0,1.00,1.00,1.01,1
7036,26.4,1.01,12.70,101.00,25.90,21.90,30.5,392.0,13.30,2996.00,18.90,1
7064,58.3,1.58,24.80,61.00,18.40,196.00,284.0,4573.0,1.00,3.49,1.53,1
7100,29.4,2.69,8.98,109.00,43.30,8.20,16.4,233.0,1.05,6098.00,25.90,1


In [29]:
df_iqr_outliers

,raf,mek,plc,pip2,pip3,erk,akt,pka,pkc,p38,jnk,anomaly
359,53.3,34.60,2.89,1.11,764.00,8.66,12.7,219.0,24.60,63.8,137.00,1
480,85.8,62.10,53.30,325.00,239.00,12.90,24.6,403.0,10.60,14.3,12.20,1
671,63.2,37.20,37.90,81.30,39.20,316.00,621.0,1472.0,18.10,30.8,9.73,1
748,37.5,16.40,18.60,41.00,5.19,2571.00,3555.0,4491.0,14.60,36.2,24.40,1
904,71.7,40.00,27.10,47.00,219.00,14.50,25.3,939.0,12.70,23.9,63.20,1
...,...,...,...,...,...,...,...,...,...,...,...,...
7141,61.5,1.78,132.00,241.00,136.00,15.10,18.6,276.0,1.49,2247.0,2.15,1
7213,14.1,1.00,22.10,79.10,13.30,7.91,10.0,368.0,2.05,1843.0,1.09,1
7298,58.3,10.40,28.60,25.70,14.20,29.70,27.6,237.0,24.10,2017.0,1.22,1
7337,58.8,1.81,12.70,231.00,35.20,9.31,13.7,302.0,63.20,2525.0,12.60,1
